# HW6. Оценка качества и сборка различных технологий

In [ ]:
!wc -m data/qc/reference.fasta

### Reads quality check

In [ ]:
!fastqc -o ./data/qc/fastqc/ "data/qc/pacbio_10x.fq.gz" "data/qc/pacbio_20x.fq.gz" "data/qc/pacbio_40x.fq.gz" "data/qc/pacbio_80x.fq.gz" "data/qc/illumina.100x.1.fq.gz" "data/qc/illumina.100x.2.fq.gz"

In [ ]:
!Trimmomatic PE "data/qc/illumina.100x.1.fq.gz" "data/qc/illumina.100x.2.fq.gz" -baseout "data/qc/illumina_trimmed" LEADING:10 TRAILING:10 SLIDINGWINDOW:10:10 MINLEN:20

## Long read assembly with flye

In [ ]:
!flye --pacbio-raw data/qc/pacbio_80x.fq.gz --out-dir data/qc/pb_80x --genome-size 5m --threads 4

## Short read assembly with SPAdes

In [ ]:
!spades.py -1 ./data/qc/illumina_trimmed_1P.fq -2 ./data/qc/illumina_trimmed_2P.fq -o ./data/qc/spades_illumina 

## Hybrid assembly with SPAdes

In [ ]:
!spades.py -1 ./data/qc/illumina_trimmed_1P.fq -2 ./data/qc/illumina_trimmed_2P.fq --pacbio ./data/qc/pacbio_40x.fq.gz -o ./data/qc/spades_hybrid_40x

## Quast

In [ ]:
!python3 ~/Prog/tools/quast-5.0.2/quast.py -r data/qc/reference.fasta data/qc/spades_hybrid_10x/contigs.fasta data/qc/spades_hybrid_20x/contigs.fasta data/qc/spades_hybrid_40x/contigs.fasta data/qc/spades_illumina/contigs.fasta data/qc/pb_20x/assembly.fasta data/qc/pb_40x/assembly.fasta data/qc/pb_80x/assembly.fasta

<img src="images/quast_comp.png">

## Results

По результатам QUAST можно сравнить сборку коротких ридов Illumina, длинных ридов PacBio и гибридную сборку.  
1) Использование длинных ридов позволяет значительно уменьшить % непокрытых участков с 2% до 0.1%  
2) Что касается точности, сборки из коротких ридов Illumina все еще вне конкуренции.  
3) Для длинных ридов критически важно хорошее покрытие, при этом огромные улучшения с точки зрения LGA50, NGA50.  
4) Гибридные сборки хорошо себя проявляют с точки зрения баланса: хорошее покрытие, мало инделов, значительно более хорошие показатели LGA50, NGA50 по сравнению с короткими ридами.  
5) Если есть возможность использовать длинные риды с высоким покрытием, стоит так делать - лучшие рещультаты на PacBio 80x (не зря Next-Next Gen).  